In [1]:
# import sys
# sys.path.append('/Users/c/Desktop/Desktop_Funnel/anaconda3/lib/python3.9/site-packages')

import multiprocess as mp
from multiprocessing import Manager
from threading import Lock

from copy import deepcopy
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('png')

from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from gerrychain.tree import bipartition_tree
"""from GerryChainMain.gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from GerryChainMain.gerrychain.proposals import recom
from GerryChainMain.gerrychain.tree import bipartition_tree"""
from PIL import Image
import pandas
from functools import partial

In [2]:
def get_updaters():
    """
    :return: updaters
    
    Does not need to be called. Will be used by generate_partition.
    """
    elections = [
        Election("REP22_PARTY", {"Democratic": "PARTY_DEM", "Republican": "PARTY_REP"}),
        # UNK and HISP included to complete distribution. Two can be merged.
        Election("REP22_RACE", {"White": "ETH1_EUR", "Black": "ETH1_AA", "Asian": "ETH1_ESA", 
                               "Hispanic": "ETH1_HISP", "Unknown": "ETH1_UNK"}) 
    ]
    
    # Population updater, for computing how close to equality the district
    # populations are. "TOTPOP" is the population column from our shapefile.
    my_updaters = {"population": updaters.Tally("POPULATION", alias="population")}
    
    # Election updaters, for computing election results using the vote totals
    # from our shapefile.
    election_updaters = {election.name: election for election in elections}
    my_updaters.update(election_updaters)
    
    return my_updaters

In [3]:
def generate_partition(file_path, iters):
    """
    :param file_path: file_path to the shape file to generate partition from
    :param iters: amount of recom steps this algorithm should take
    :return: the final partition
    
    Creates a random partition object
    """
    
    graph = Graph.from_file(file_path)

    my_updaters = get_updaters()
    
    initial_partition = GeographicPartition(
        graph, 
        assignment="DISTRICT", 
        updaters=my_updaters
    )

    ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)
    proposal = partial(
        recom,
        pop_col="POPULATION",
        pop_target=ideal_population,
        epsilon=0.02,
        node_repeats=2,
        method=partial(
            bipartition_tree,
            max_attempts=100,
            allow_pair_reselection=True
        )
    )
    
    compactness_bound = constraints.UpperBound(
        lambda p: len(p["cut_edges"]),
        2*len(initial_partition["cut_edges"])
    )

    pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.1)
    
    chain = MarkovChain(
        proposal=proposal,
        constraints=[
            pop_constraint,
            compactness_bound
        ],
        accept=accept.always_accept,
        initial_state=initial_partition,
        total_steps=iters
    )
    
    for partition in chain.with_progress_bar():
        pass
    
    return partition
    # shared_data['partition'] = partition
    # return None

In [4]:
# single run test
# partition = generate_partition("DE_precincts.geojson", iters=1000).val

In [5]:
# print(partition)

In [6]:
# print(partition['REP22_PARTY'].percents("Democratic"))

In [7]:
def get_percent_districts(partition, race, percent=.5):
    districts = np.array(partition['REP22_RACE'].percents(race))
    return np.where(districts > percent, 1 , 0)

In [8]:
# just a sanity check to compare with other data to make sure everything is in order
"""percent = .44
print(get_percent_districts("Black", percent))
print()
de = gpd.read_file('DE_precincts.geojson')
group_de = de.groupby("DISTRICT")
summed_black_pop = group_de['ETH1_AA'].sum()/group_de['TOTAL_REG'].sum()
print(summed_black_pop > percent)"""
# results are consistent (regions are in order)

'percent = .44\nprint(get_percent_districts("Black", percent))\nprint()\nde = gpd.read_file(\'DE_precincts.geojson\')\ngroup_de = de.groupby("DISTRICT")\nsummed_black_pop = group_de[\'ETH1_AA\'].sum()/group_de[\'TOTAL_REG\'].sum()\nprint(summed_black_pop > percent)'

In [9]:
# box_plots = {'White':[], 'Black':[], 'Asian':[], 'Republican':[], 'Democratic':[]}
thresholds = [.37, .5, .6] 
races = ['White', 'Black', 'Asian']
parties = ['Republican', 'Democratic']

init1 = {'White':float('-inf'), 'Black':float('-inf'), 'Asian':float('-inf')}
init2 = {'White':float('inf'), 'Black':float('inf'), 'Asian':float('inf')}
init3 = {'White':None, 'Black':None, 'Asian':None}

max_opportunity_district_vals = []
min_opportunity_district_vals = []
max_opportunity_districts = []
min_opportunity_districts = []

def initialize_globals():
    """
    :return: None
    
    Sets the values of necessary data trackers (for tracking data across multiple random partitions).
    """
    
    
    global box_plots, init1, init2, init3, max_opportunity_district_vals
    global min_opportunity_district_vals, max_opportunity_districts, min_opportunity_districts
    
    box_plots = {'White':[], 'Black':[], 'Asian':[], 'Republican':[], 'Democratic':[], 'points':dict()}

    """
    Data in the following is formatted as such: the first indexer (for the list) refers to which threshold
    we would like to get the value for. The second indexer is for a dictionary and should be 
    "White", "Black", "Asian", "Republican", "Democratic." This selects which group you would like to 
    get the value for.
    """
    max_opportunity_district_vals = [deepcopy(init1) for _ in range(len(thresholds))]
    min_opportunity_district_vals = [deepcopy(init2) for _ in range(len(thresholds))]
    max_opportunity_districts = [deepcopy(init3) for _ in range(len(thresholds))]
    min_opportunity_districts = [deepcopy(init3) for _ in range(len(thresholds))]

In [10]:
"""manager = None
class Manager_Wrapper():
    def __init__(self, manager, lock):
        self.manager = manager
        self.lock = lock
    
    def initialize_data(self):
        global thresholds, races, parties, init1, init2, init3
        manager = self.manager

        box_plots = manager.dict()
        box_plots['White'] = []
        box_plots['Black'] = []
        box_plots['Asian'] = []
        box_plots['Republican'] = []
        box_plots['Democratic'] = []

        init1 = manager.dict(init1)
        init2 = manager.dict(init2)
        init3 = manager.dict(init3)

        max_opportunity_district_vals = manager.list([init1.copy() for _ in range(len(thresholds))])
        min_opportunity_district_vals = manager.list([init2.copy() for _ in range(len(thresholds))])
        max_opportunity_districts = manager.list([init3.copy() for _ in range(len(thresholds))])
        min_opportunity_districts = manager.list([init3.copy() for _ in range(len(thresholds))])
        
        self.box_plots = box_plots
        self.max_opportunity_district_vals = max_opportunity_district_vals
        self.min_opportunity_district_vals = min_opportunity_district_vals
        self.max_opportunity_districts = max_opportunity_districts
        self.min_opportunity_districts = min_opportunity_districts

        return box_plots, max_opportunity_district_vals, min_opportunity_district_vals, \
                max_opportunity_districts, min_opportunity_districts


    def get_members(self):
        return self.box_plots, self.max_opportunity_district_vals,\
            self.min_opportunity_district_vals, self.max_opportunity_districts, \
            self.min_opportunity_districts
    
    def get_manager(self):
        return self.manager
    
    def get_lock(self):
        return self.lock"""

""" def initialize_data(manager):
    box_plots = manager.dict()
    box_plots['White'] = []
    box_plots['Black'] = []
    box_plots['Asian'] = []
    box_plots['Republican'] = []
    box_plots['Democratic'] = []

    # init1_dict = manager.dict({'White': float('-inf'), 'Black': float('-inf'), 'Asian': float('-inf')})
    # init2_dict = manager.dict({'White': float('inf'), 'Black': float('inf'), 'Asian': float('inf')})
    # init3_dict = manager.dict({'White': None, 'Black': None, 'Asian': None})

    # max_opportunity_district_vals = manager.list([init1_dict.copy() for _ in range(len(thresholds))])
    max_opportunity_district_vals = manager.list()
    # min_opportunity_district_vals = manager.list([init2_dict.copy() for _ in range(len(thresholds))])
    min_opportunity_district_vals = manager.list()
    # max_opportunity_districts = manager.list([init3_dict.copy() for _ in range(len(thresholds))])
    max_opportunity_districts = manager.list()
    # min_opportunity_districts = manager.list([init3_dict.copy() for _ in range(len(thresholds))])
    min_opportunity_districts = manager.list()
    
    for _ in range(len(thresholds)):
        max_opportunity_district_vals.append(manager.dict(
            {'White': float('-inf'), 'Black': float('-inf'), 'Asian': float('-inf')}))
        min_opportunity_district_vals.append(manager.dict(
            {'White': float('inf'), 'Black': float('inf'), 'Asian': float('inf')}))
        max_opportunity_districts.append(manager.dict({'White': None, 'Black': None, 'Asian': None}))
        min_opportunity_districts.append(manager.dict({'White': None, 'Black': None, 'Asian': None}))

    return box_plots, max_opportunity_district_vals, min_opportunity_district_vals, \
    max_opportunity_districts, min_opportunity_districts"""

" def initialize_data(manager):\n    box_plots = manager.dict()\n    box_plots['White'] = []\n    box_plots['Black'] = []\n    box_plots['Asian'] = []\n    box_plots['Republican'] = []\n    box_plots['Democratic'] = []\n\n    # init1_dict = manager.dict({'White': float('-inf'), 'Black': float('-inf'), 'Asian': float('-inf')})\n    # init2_dict = manager.dict({'White': float('inf'), 'Black': float('inf'), 'Asian': float('inf')})\n    # init3_dict = manager.dict({'White': None, 'Black': None, 'Asian': None})\n\n    # max_opportunity_district_vals = manager.list([init1_dict.copy() for _ in range(len(thresholds))])\n    max_opportunity_district_vals = manager.list()\n    # min_opportunity_district_vals = manager.list([init2_dict.copy() for _ in range(len(thresholds))])\n    min_opportunity_district_vals = manager.list()\n    # max_opportunity_districts = manager.list([init3_dict.copy() for _ in range(len(thresholds))])\n    max_opportunity_districts = manager.list()\n    # min_opportunity_

In [11]:
"""def handle_partition(file_path,
                     partition_steps,
                     lock,
                     box_plots,
                     max_opportunity_district_vals,
                     min_opportunity_district_vals,
                     max_opportunity_districts,
                     min_opportunity_districts, partition=None):"""
def handle_partition(file_path, partition_steps, partition=None):
    """
    :param file_path: the file path to be read. if partition is not None, this field is ignored
    :param partition_steps: the steps in our partition. if partition is not None this field is ignored
    :param partition: the partition we want to handle. None or a Partition Object.
    :return: None
    
    This performs necessary post(mid)-processing of random partitions. It will add it to boxplot data
    and if the partition is "of note" will be saved.
    """
    
    global races, parties
    """box_plots, max_opportunity_district_vals, min_opportunity_district_vals, \
            max_opportunity_districts, min_opportunity_districts = manager.get_members()
    lock = manager.get_lock()"""
    global box_plots, max_opportunity_district_vals, min_opportunity_district_vals, \
            max_opportunity_districts, min_opportunity_districts
    
    if partition is None:
        partition = generate_partition(file_path, partition_steps)
    
    # print(box_plots['White'])
    # print(partition['REP22_RACE'])
    
    # lock.acquire()
    # print("\nENTERED")
    for race in races:
        box_plots[race].append(sorted(partition['REP22_RACE'].percents(f'{race}')))
        for i, threshold in enumerate(thresholds):
            total_opportunity_districts = np.sum(get_percent_districts(partition, race, percent=threshold))

            # perform checks to see if the partition should be saved
            if total_opportunity_districts > max_opportunity_district_vals[i][race]:
                # print(f"MAX_{race} {max_opportunity_district_vals[i][race]} -> {total_opportunity_districts}")
                # print(max_opportunity_district_vals[i][race])
                max_opportunity_district_vals[i][race] = total_opportunity_districts
                # print("x")
                # print(partition.graph.graph)
                max_opportunity_districts[i][race] = partition 
                # print(max_opportunity_district_vals[i][race])

            if total_opportunity_districts < min_opportunity_district_vals[i][race]:
                # print(f"MIN_{race} {min_opportunity_district_vals[i][race]} -> {total_opportunity_districts}")
                # print(max_opportunity_district_vals[i][race])
                min_opportunity_district_vals[i][race] = total_opportunity_districts
                # print("x")
                # print(partition.graph.graph)
                min_opportunity_districts[i][race] = partition 
                # print(min_opportunity_district_vals[i][race])

    for party in parties:
        box_plots[party].append(sorted(partition['REP22_PARTY'].percents(party)))

    # print(max_opportunity_district_vals)
    # print(max_opportunity_districts)
    # print(min_opportunity_district_vals)
    # print(min_opportunity_districts)
    # print("EXIT\n")
    # lock.release()

In [12]:
def generate_box_plots(data, state):
    """
    :param data: pd.DataFrame to generate data from
    :param state: String of state name
    :return: boxplot data generated
    """
    districts = []

    def get_bounds(q1, q3, col, bound=1.5):
        iqr = q3 - q1
        lower_bound = q1 - (bound * iqr)
        upper_bound = q3 + (bound * iqr)
        filtered_col = col[(col > lower_bound) & (col < upper_bound)]
        return filtered_col.min(), filtered_col.max()


    for col in data.columns:
        lower = data[col].quantile(0.25)
        upper = data[col].quantile(.75)
        lower_bound, upper_bound = get_bounds(lower, upper, data[col])

        district = {}
        district['q1'] = lower
        district['median'] = data[col].quantile(.5)
        district['q3'] = upper
        district['min'] = lower_bound
        district['max'] = upper_bound
        district['state'] = state
        districts.append(district)
    
    return districts

In [13]:
def get_ensemble(file_path, state, partition_steps, iterations):
    """
    :param file_path: file_path of the geojson to be used
    :param state: The name of the state we are creating an ensemble for
    :param partition_steps: iterations in each partition generated
    :param iterations: amount of partitions to generate
    :return: partitions of note and stats.
    """
    """manager = Manager()
    lock = manager.Lock()
    
    box_plots, max_opportunity_district_vals, min_opportunity_district_vals, \
            max_opportunity_districts, min_opportunity_districts = initialize_data(manager)
    num_processes = 3 # mp.cpu_count()
    
    # try:
    # Create a pool of worker processes
    pool = mp.Pool(processes=num_processes)

    file_paths = [(file_path, partition_steps, lock, box_plots, 
                   max_opportunity_district_vals, 
                   min_opportunity_district_vals, 
                   max_opportunity_districts, 
                   min_opportunity_districts)] * iterations
    print(file_paths)

    # Use pool.map to apply the square function to each number in parallel
    pool.starmap(handle_partition, file_paths)"""
    global box_plots, max_opportunity_district_vals, min_opportunity_district_vals, \
                   max_opportunity_districts, min_opportunity_districts
    initialize_globals()
    for i in range(iterations):
        handle_partition(file_path, partition_steps)
    
    """except Exception as e:
        pool.close()
        print(e)
        return"""

    # Close the pool
    # pool.close()
    # pool.join()
    
    # print(max_opportunity_districts)
    # print(min_opportunity_districts)
    
    ensemble_box_plots = {}
    for key,value in box_plots.items():
        # boxplot data
        ensemble_box_plots[key] = generate_box_plots(pd.DataFrame(value), state)
    
    initial_partition = generate_partition(file_path, 0) # initial partition
    
    
    # add initial partition points for box plot
    for party in parties:
        box_plots['points'][f'initial_partition_{party}'] = \
            sorted(initial_partition["REP22_PARTY"].percents(party))
    for race in races:
        box_plots['points'][f'initial_partition_{race}'] = \
            sorted(initial_partition["REP22_RACE"].percents(race))
    
    # print(max_opportunity_districts)
    # print(min_opportunity_districts)
    
    # points for other saved partitions
    # gets the points for a boxplots of a specific partition that is interesting for a specific race and threshold
    # getting the boxplot means getting boxplot data for "all" data about the partition
    for race1 in races:
        for i, threshold in enumerate(thresholds):
            # points_max = {}
            # points_min = {}
            for party in parties:
                box_plots['points'][f'max_{race1}_for_{party}_@{threshold}'] = \
                        sorted(max_opportunity_districts[i][race1]["REP22_PARTY"].percents(party))
                box_plots['points'][f'min_{race1}_for_{party}_@{threshold}'] = \
                        sorted(min_opportunity_districts[i][race1]["REP22_PARTY"].percents(party))
            for race2 in races:
                box_plots['points'][f'max_{race1}_for_{race2}_@{threshold}'] = sorted(max_opportunity_districts[i][race1]["REP22_RACE"].percents(race2))
                box_plots['points'][f'min_{race1}_for_{race2}_@{threshold}'] = sorted(min_opportunity_districts[i][race1]["REP22_RACE"].percents(race2))
            # box_plots['points'][f'max_{race1}_for_{party}_@{threshold}'] = points_max
            # box_plots['points'][f'min_{race1}_for_{race2}_@{threshold}'] = points_min

    # currently includes only opportunity districts and seats won
    # The way stats is structed is the following:
    #       Which "Threshold" we would like to check
    #       The group this partition was interesting for (some race)
    #       The group you would like to plot (some race/political group)
    #       The reason this partition was interesting for: min, max, or initial
    stats = []
    for i in range(len(thresholds)):
        mini_stat = {'White': {'min':{}, 'max':{}, 'initial_partition':{}}, 
             'Asian': {'min':{}, 'max':{}, 'initial_partition':{}}, 
             'Black': {'min':{}, 'max':{}, 'initial_partition':{}},
             'Republican': {'min':{}, 'max':{}, 'initial_partition':{}}, 
             'Democratic': {'min':{}, 'max':{}, 'initial_partition':{}}}
        stat = {}
        for race in races:
            stat[race] = deepcopy(mini_stat)
        stats.append(stat)
    
    
    # partition is the opportunity districts at a specific threshold to save 
    def save_stats(partition, inner_name, race1, i, threshold, stats):    
        election = partition["REP22_RACE"]
        for race2 in races: # opportunity districts
            # array of winning regions
            opportunity_regions = [election.totals_for_party[race2][region]/election.totals[region] 
                                                            > threshold for region in election.regions]
            ttl_opportunity_districts = np.sum(opportunity_regions)
            stats[i][race1][race2][inner_name][f'reg'] = opportunity_regions
            stats[i][race1][race2][inner_name][f'ttl'] = ttl_opportunity_districts

        election = partition["REP22_PARTY"]
        for party in parties: # seats won
            winning_regions = [election.won(party, region) for region in election.regions]
            ttl_wins = np.sum(winning_regions)
            stats[i][race1][party][inner_name][f'reg'] = winning_regions
            stats[i][race1][party][inner_name][f'ttl'] = ttl_wins
    
    
    # calculate statistics for initial partition (opportunity districts)
    for i, threshold in enumerate(thresholds):
        for race in races:
            save_stats(initial_partition, 'initial_partition', race, i, threshold, stats)

    # calculate statistics for saved partitions
    for i, threshold in enumerate(thresholds):
        for race in races:
            save_stats(min_opportunity_districts[i][race], 'min', race, i, threshold, stats)
            save_stats(max_opportunity_districts[i][race], 'max', race, i, threshold, stats)

    # THESE ARE NOT SHAPEFILES. IF SHAPEFILES ARE DESIRED YOU MUST CONVERT THEM SERPARATELY OUTSIDE
    saved_partitions = {'min':min_opportunity_districts, 'max':max_opportunity_districts, 
                        'initial':initial_partition}
    
    # return saved items
    return saved_partitions, stats

In [14]:
partitions, stats = get_ensemble('DE_precincts.geojson', 'Delaware', 500, 3)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [15]:
max_opportunity_district_vals

[{'White': 38, 'Black': 9, 'Asian': 0},
 {'White': 31, 'Black': 2, 'Asian': 0},
 {'White': 27, 'Black': 1, 'Asian': 0}]

In [16]:
min_opportunity_district_vals

[{'White': 37, 'Black': 6, 'Asian': 0},
 {'White': 30, 'Black': 0, 'Asian': 0},
 {'White': 24, 'Black': 0, 'Asian': 0}]

In [17]:
box_plots

{'White': [[0.120918024781786,
   0.20016309120562037,
   0.3601080198455065,
   0.37074568453420925,
   0.40931023138502026,
   0.41883904684875256,
   0.41903467029231817,
   0.4407078605444177,
   0.44411573279687594,
   0.47828578706062147,
   0.5076497695852534,
   0.5502584721424468,
   0.5759099480029712,
   0.5940676952517642,
   0.6037083804080547,
   0.6116402116402117,
   0.6120729045032238,
   0.6171219838452436,
   0.6296221560813197,
   0.6325846251186333,
   0.6381863380890618,
   0.6661109329913434,
   0.6728362726267388,
   0.6739310059904978,
   0.7094371324558947,
   0.7158262170417005,
   0.7211252839419885,
   0.7269830622170048,
   0.7430318288077684,
   0.7515177110278203,
   0.7563105117565698,
   0.7600055393989752,
   0.772800088383141,
   0.7806416232519879,
   0.7892492942328744,
   0.7929316888045541,
   0.8056847838420516,
   0.8254878387126607,
   0.8303278319917256,
   0.8402285082465678,
   0.8424377004687885],
  [0.10922962411611463,
   0.3376745732653

In [18]:
partitions

{'min': [{'White': <GeographicPartition [41 parts]>,
   'Black': <GeographicPartition [41 parts]>,
   'Asian': <GeographicPartition [41 parts]>},
  {'White': <GeographicPartition [41 parts]>,
   'Black': <GeographicPartition [41 parts]>,
   'Asian': <GeographicPartition [41 parts]>},
  {'White': <GeographicPartition [41 parts]>,
   'Black': <GeographicPartition [41 parts]>,
   'Asian': <GeographicPartition [41 parts]>}],
 'max': [{'White': <GeographicPartition [41 parts]>,
   'Black': <GeographicPartition [41 parts]>,
   'Asian': <GeographicPartition [41 parts]>},
  {'White': <GeographicPartition [41 parts]>,
   'Black': <GeographicPartition [41 parts]>,
   'Asian': <GeographicPartition [41 parts]>},
  {'White': <GeographicPartition [41 parts]>,
   'Black': <GeographicPartition [41 parts]>,
   'Asian': <GeographicPartition [41 parts]>}],
 'initial': <GeographicPartition [41 parts]>}

In [19]:
stats

[{'White': {'White': {'min': {'reg': [False,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      False,
      True,
      True,
      True,
      True,
      True,
      False,
      False,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True],
     'ttl': 37},
    'max': {'reg': [True,
      False,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      False,
      False,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
      True,
     

In [20]:
"""manager = Manager()
    lock = manager.Lock()
    file_path = 'DE_precincts.geojson'
    partition_steps = 500
    # box_plots, max_opportunity_district_vals, min_opportunity_district_vals, \
                # max_opportunity_districts, min_opportunity_districts = initialize_data(manager)
    box_plots = {}
    box_plots['White'] = []
    box_plots['Black'] = []
    box_plots['Asian'] = []
    box_plots['Republican'] = []
    box_plots['Democratic'] = []

    # init1_dict = manager.dict({'White': float('-inf'), 'Black': float('-inf'), 'Asian': float('-inf')})
    # init2_dict = manager.dict({'White': float('inf'), 'Black': float('inf'), 'Asian': float('inf')})
    # init3_dict = manager.dict({'White': None, 'Black': None, 'Asian': None})

    # max_opportunity_district_vals = manager.list([init1_dict.copy() for _ in range(len(thresholds))])
    max_opportunity_district_vals = []
    # min_opportunity_district_vals = manager.list([init2_dict.copy() for _ in range(len(thresholds))])
    min_opportunity_district_vals = []
    # max_opportunity_districts = manager.list([init3_dict.copy() for _ in range(len(thresholds))])
    max_opportunity_districts = []
    # min_opportunity_districts = manager.list([init3_dict.copy() for _ in range(len(thresholds))])
    min_opportunity_districts = []

    for _ in range(len(thresholds)):
        max_opportunity_district_vals.append(manager.dict(
            {'White': float('-inf'), 'Black': float('-inf'), 'Asian': float('-inf')}))
        min_opportunity_district_vals.append(manager.dict(
            {'White': float('inf'), 'Black': float('inf'), 'Asian': float('inf')}))
        max_opportunity_districts.append(manager.dict({'White': None, 'Black': None, 'Asian': None}))
        min_opportunity_districts.append(manager.dict({'White': None, 'Black': None, 'Asian': None}))

    handle_partition(file_path, partition_steps, lock, box_plots, 
                       max_opportunity_district_vals, 
                       min_opportunity_district_vals, 
                       max_opportunity_districts, 
                       min_opportunity_districts)"""

"manager = Manager()\n    lock = manager.Lock()\n    file_path = 'DE_precincts.geojson'\n    partition_steps = 500\n    # box_plots, max_opportunity_district_vals, min_opportunity_district_vals,                 # max_opportunity_districts, min_opportunity_districts = initialize_data(manager)\n    box_plots = {}\n    box_plots['White'] = []\n    box_plots['Black'] = []\n    box_plots['Asian'] = []\n    box_plots['Republican'] = []\n    box_plots['Democratic'] = []\n\n    # init1_dict = manager.dict({'White': float('-inf'), 'Black': float('-inf'), 'Asian': float('-inf')})\n    # init2_dict = manager.dict({'White': float('inf'), 'Black': float('inf'), 'Asian': float('inf')})\n    # init3_dict = manager.dict({'White': None, 'Black': None, 'Asian': None})\n\n    # max_opportunity_district_vals = manager.list([init1_dict.copy() for _ in range(len(thresholds))])\n    max_opportunity_district_vals = []\n    # min_opportunity_district_vals = manager.list([init2_dict.copy() for _ in range(len(th